# Cloud Expenditure Optimization – Notebook Suite
This set of notebooks follows the architecture: ETL → Database → ML (Failure, Cost) → Dashboards.

**Data input**: `../data/sample_reports_100.csv` (or `../data/sample_reports.csv`)

**Outputs**: cleaned data and artifacts in `../results/`.

## 01 – ETL & Exploration
Load synthetic reports, clean/transform, and save `cleaned_reports.csv`.

In [ ]:
import os
from pathlib import Path
import pandas as pd

# Locate data (tries multiple paths to be repo-friendly)
possible_paths = [
    '../data/sample_reports_100.csv',
    '../data/sample_reports.csv',
    '/mnt/data/sample_reports_100.csv',
    '/mnt/data/sample_reports.csv'
]
data_path = next((p for p in possible_paths if Path(p).exists()), None)
assert data_path is not None, f'Could not find CSV. Checked: {possible_paths}'
print('Using data file:', data_path)

df = pd.read_csv(data_path, parse_dates=['timestamp'])
df.head()


In [ ]:
# Basic profiling
print(df.shape)
print(df.dtypes)
df.isna().sum()


In [ ]:
# Standardize columns and basic cleaning
df.columns = [c.strip().lower() for c in df.columns]
df = df.sort_values('timestamp').reset_index(drop=True)

# Ensure numeric
num_cols = ['response_time_ms', 'cpu_usage', 'memory_usage', 'cost_usd']
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# Fill missing numeric with median; categorical with mode
df[num_cols] = df[num_cols].fillna(df[num_cols].median())
for c in ['system_name', 'error_code', 'status']:
    if c in df.columns:
        df[c] = df[c].fillna(df[c].mode()[0])

df.describe(include='all')


In [ ]:
# Simple feature engineering examples
df['hour'] = df['timestamp'].dt.hour
df['day'] = df['timestamp'].dt.date.astype(str)
df['is_peak'] = (df['hour'].between(9, 18)).astype(int)

# Save cleaned dataset next to data folder or into /mnt/data
out_candidates = ['../data/cleaned_reports.csv', '/mnt/data/cleaned_reports.csv']
out_path = next((p for p in out_candidates if Path(p).parent.exists()), out_candidates[0])
df.to_csv(out_path, index=False)
print('Saved cleaned dataset to:', out_path)


In [ ]:
# Quick visual sanity checks (no seaborn, single-plot rule)
import matplotlib.pyplot as plt

df_daily = df.groupby('day', as_index=False)['cost_usd'].sum()
plt.figure()
plt.plot(df_daily['day'], df_daily['cost_usd'])
plt.title('Daily Cloud Cost (Synthetic)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
